In [1]:
from numba import jit
from scipy.integrate import solve_ivp

%run ../Utilities/Utilities.ipynb

In [ ]:
#Initial values in task 2c
def f_2c(x):
    return np.exp(-400 * (x-1/2)**2 )

# Heat Equation solver class.

In [1]:
class heatEqSolver:
    '''
    Solver object for the 1D heat eqn with double neumann boundary conditions. Upon initialization, the matrices for forward
    Euler, backward Euler and Crank-Nicolson are initialized.
    
    M       : (int) number of points in x-direction
    k       : (float) stepsize in t-direction
    U0      : (ndarray) inital temperature as a fnc of x
    U_x0    : (float) Value of the x-derivative in the left endpoint
    U_x1    : (float) value of the x-derivative in the right endpoint
    
    returns : heatEqSolver object
    '''
    def __init__(self, M, k, U0, U_x0 = 0, U_x1 = 0):
        self.M = M
        self.k = k 
        self.U0 = U0
        h = 1/(self.M + 1)
        
        self.boundaryConditions = np.array([U_x0, U_x1])
        
        ## +++++ Make matrix for eulers method
        
        #Make superdiagonal
        sup = np.ones(self.M+1)
        sup[0]=0 

        #Make diagonal
        diag =h**2/self.k*np.ones(self.M+2) - 2*np.ones(self.M+2)
        diag[0]=0 
        diag[-1]=0 

        #Make subdiagonal
        sub = np.ones(self.M+1)
        sub[-1]=0 
        
        #Construct sparse matrix
        self.eulerA = self.k/h**2 * diags([sup,diag,sub], offsets=[1,0,-1], format = "csc")
        
        ## ----- Make matrix for eulers method
        
        
        
        ## +++++ Make matrix for crank-nicholson
         
        self.r = self.k/(1/(self.M+1))**2

        #Make superdiagonal
        sup = -self.r/2 * np.ones(self.M+1)
        sup[0] = -4/3
        
        #Make supersuperdiagonal
        supsup = np.zeros(self.M)
        supsup[0] = 1/3

        #Make diagonal
        diag = (1+self.r)*np.ones(self.M+2)
        diag[0] = 1
        diag[-1] = 1

        #Make subdiagonal
        sub = -self.r/2 * np.ones(self.M+1)
        sub[-1] = -4/3
        
        #Make subsub diag
        subsub = np.zeros(self.M)
        subsub[-1] = 1/3
        
        #Construct sparse matrix
        crankA = diags([supsup,sup, diag, sub, subsub], offsets = [2,1, 0, -1, -2], format = "csc")
        
        #Perform lu-factorization to improve computation time
        self.sluCrank = sl.splu(crankA)
        
        ## ----- Make matrix for crank-nicholson
        
        
        ## +++++ Make matrix for backwards euler
        
        #Make supersuperdiagonal
        supsup = np.zeros(M)
        supsup[0] = -1/(2*h)
        
        #Make superdiagonal
        sup = -self.r*np.ones(M+1)
        sup[0] = 2/h
        
        #Make diagonal
        diag = (1+2*self.r)*np.ones(M+2)
        diag[0] = -3/(2*h)
        diag[-1] = 3/(2*h)
        
        #Make subdiagonal
        sub = -self.r*np.ones(M+1)
        sub[-1] = -2/h
        
        #Make subsubdiagonal
        subsub = np.zeros(M)
        subsub[-1] = 1/(2*h)
        
        #Construct sparse matrix
        backwardEulerA = diags([supsup,sup, diag, sub, subsub], offsets = [2,1, 0, -1, -2], format = "csc")
        
        #Perform LU-factorization to save time
        self.sluBackEuler = sl.splu(backwardEulerA)
        
        ## ----- Make matrix for backwards euler
    
    #Make sparse method
    def euler_IBVP(self, U):
        #Compute next time step
        U = self.eulerA*U
        
        #Find the missing values on the end points
        U[0]=4/3*U[1]-1/3*U[2] + 2/3*self.boundaryConditions[0]/(self.M + 1)
        U[-1]=4/3*U[-2]-1/3*U[-3] + 2/3*self.boundaryConditions[1]/(self.M + 1)
        
        return U
    
    def Backward_Euler(self, U):
        b = np.copy(U)
        b[0] = self.boundaryConditions[0]
        b[-1] = self.boundaryConditions[1]
        
        return self.sluBackEuler.solve(b)
    
    #Solves it with a sparse matrix
    def crank_nicholson(self, U):
        #Defining d
        d = lambda m : self.r/2 * U[m-1] + (1-self.r) * U[m] + self.r/2 * U[m+1]

        #Making D vector
        D = np.zeros_like(U)
        for mm in range(1,len(U)-1):
            D[mm] = d(mm)
            
        D[0] = 2/3*self.boundaryConditions[0]/(self.M + 1)
        D[-1] = 2/3*self.boundaryConditions[1]/(self.M + 1)
        
        U = self.sluCrank.solve(D)
        
        return U
        
    
    def solve(self, method, timesteps, M = None, k = None, U0 = None):
        #Fetching parameters
        if M == None:
            M = self.M
            
        if k == None:
            k = self.k
            
        if U0 == None:
            U0 = self.U0
            
        self.timesteps = timesteps
        
        #Selecting method
        method = method.lower()
        if method == "euler":
            method_func = self.euler_IBVP
        elif method in ["backward euler", "be"]:
            method_func = self.Backward_Euler
        elif method in ["crank", "crank nicholson"]:
            method_func = self.crank_nicholson  
        else:
            raise Exception("Invalid method specified")
        
        #Initialize solution
        self.U = np.zeros((self.M+2, timesteps+1))
        
        #Initial temperature function
        self.U[:, 0] = self.U0
        
        #Start a timer
        startTime = time.time()
        
        #Run timesteps iterations
        for n in range(1,timesteps+1):
            if n%1000 == 0:
                sys.stdout.write("\rCalculating using " + method + " and with m = " + str(self.M) + ":  " + str(round(n/timesteps*100, 1)) + "%     Estimated time remaining: " + str(round((time.time()-startTime)*timesteps/(n+1) - (time.time()-startTime))) + " seconds        ")
                sys.stdout.flush()
            self.U[:, n] = method_func(self.U[:,n-1])
            
        sys.stdout.write("\rCalculation using " + method + " done.           Elapsed time: " + str(round(time.time() - startTime)) + " seconds                                                      ")
        sys.stdout.flush()
        print(" ")

## UMR

In [ ]:
def UMR(Ms, Ns, method, plot = False):
    '''
    Perform UMR with the specified sets of Ms and Ns
    
    Ms      : (ndarray) Array of spacial steps
    Ns      : (ndarray) Array of temporal steps
    
    returns : (dictionary) {"continous", "discrete"} L2 and l2 norm error
    '''
    
    if Ms.shape != Ns.shape:
        raise Exception("Ms and Ns not of same shape")
    
    #Initialize errors
    errorL2 = np.zeros_like(Ms, dtype="float64")
    errorl2 = np.zeros_like(Ms, dtype="float64")
    
    for i in range(len(Ms)):
        #Extract desired m and n
        m = Ms[i]
        n = Ns[i]
        
        #Compute k and h
        k = 1.0/n
        h = 1/(m+1)
        
        #Slicing because np.arange can sometimes take one step too much due to rounding errors
        x_vec = np.arange(0,1+h,h)[:int(m)+2]
        t1 =  np.ones_like(x_vec)*(1.0)
        
        #Initial temperature
        U0 = initfunc(x_vec)

        #Create a numerical solver object
        numSol = heatEqSolver(m, k, U0, 0, 0) 

        #Solve it using specified method
        numSol.solve(method, n) 
        
        #Find the analytic solution at the last time-step
        analyticSol = analyticSolution(x_vec, t1) 
        
        #Used for debugging
        if(plot):
            fig, axs = plt.subplots(1,2)
            
            axs[0].plot(x_vec, numSol.U[:,-1]-analyticSol, label="diff")
            axs[0].legend()
            axs[1].plot(x_vec, numSol.U[:,-1], label="Numerical")
            axs[1].plot(x_vec, analyticSol, label="Analytic")
            axs[1].legend()
            
            plt.show()

        #Fill error into arrays
        errorl2[i] = l2norm(numSol.U[:,-1]-analyticSol)/l2norm(analyticSol)
        errorL2[i] = integralNorm(numSol.U[:,-1]-analyticSol,x_vec)/integralNorm(analyticSol,x_vec)
        
        
    return {"continuous": errorL2, "discrete": errorl2}

# 2c) Solver for Burgers' equation

In [ ]:
#Updates the approximation of u_t at time t_n using the semi-discretization shown in our report
@jit(nopython = True)
def F_Burgers(y,F_vec,M):
    h=1/(M+1)
    
    F_vec[:] = np.zeros(M)
    F_vec[0:M-1] += y[1:]**2/(-4*h)
    F_vec[1:M] -= y[:M-1]**2/(-4*h)

In [ ]:
#Implementation of RK45
#Returns an array of u(x_m), m=1,2,...,M at different times t

@jit(nopython = True)
def solve_RK_45(t_0, T, h, y_0):
    
    # Runge-Kutta 45
    
    assert T > t_0, "T must be larger than t_0"
    assert h > 0, "h must be larger than 0"
    assert h < T - t_0, "h must be smaller than T - t_0"
    
    # Calculate number of steps with step length h required and the last step h_last
    n = np.int(np.floor((T - t_0)/h))
    h_last = (T - t_0) - n*h
    
    # Dimension of problem
    m = len(y_0)
    
    # Vector to store solutions y = (q_1, q_2, p_1, p_2), i.e x and y coordinate of space and velocity respectively
    if (h_last > 0):
        y_vec = np.zeros((n + 2, m))
    else:
        y_vec = np.zeros((n + 1, m))
    y_vec[0,:] = y_0
    
    # Vector to store intermediate calculations F_1, F_2... for Runge-Kutta and Shampine-Bogacki
    F_vec = np.zeros((m, 4))
        
    # Run the loop for every step except the very last step
    for i in range(n):
        F_Burgers(y_vec[i,:], F_vec[:,0],m)
        F_Burgers(y_vec[i,:] + 0.5 * h * F_vec[:,0], F_vec[:,1],m)
        F_Burgers(y_vec[i,:] + 0.5 * h * F_vec[:,1], F_vec[:,2],m)
        F_Burgers(y_vec[i,:] + h * F_vec[:,2], F_vec[:,3],m)
        y_vec[i+1,:] = y_vec[i,:] + h * 1.0/6.0 * (F_vec[:,0] + 2.0 * F_vec[:,1] + 2.0 * F_vec[:,2] + F_vec[:,3])
    
    # Last iteration uses different step length
    if(h_last > 0.0):
        F_Burgers(y_vec[n,:], F_vec[:,0],m)
        F_Burgers(y_vec[n,:] + 0.5 * h_last * F_vec[:,0], F_vec[:,1],m)
        F_Burgers(y_vec[n,:] + 0.5 * h_last * F_vec[:,1], F_vec[:,2],m)
        F_Burgers(y_vec[n,:] + h_last * F_vec[:,2], F_vec[:,3],m)
        y_vec[n+1,:] = y_vec[n,:] + h_last * 1.0/6.0 * (F_vec[:,0] + 2.0 * F_vec[:,1] + 2.0 * F_vec[:,2] + F_vec[:,3])
    
    return y_vec

## Plotting

In [ ]:
def errorPlots(startIndex, endIndex, constant, typ="n-const", plot = False):
    '''
    Make errorplot for the manufactured solution
    
    startindex   : (int) startindex for the mesh refinement
    endindex     : (int) endindex for the mesh refinement
    typ          : (str) type of mesh refinement: one of "n-const", "m-const", "c-const", "r-const"
    constant     : (float) the value for the paramater that is held constant
    
    returns      : None
    '''
    
    #1. Do UMR in x-direction with k = const
    if typ == "n-const":
        timesteps = constant 
        
        M = np.logspace(np.log2(startIndex), np.log2(endIndex), int(1+np.log2(endIndex) - np.log2(startIndex)), base=2, dtype="int")
        
        eulerError = UMR(M, timesteps*np.ones_like(M, dtype="int"), "be", plot=plot)
        crankError = UMR(M, timesteps*np.ones_like(M), "crank", plot=plot)
        
        dof = constant*M
        
        expectedEuler = 2
        expectedCrank = 2
    
    #2. Do UMR in t-direction, keep h-constant very smoll
    if typ == "m-const":
        
        timesteps = np.logspace(np.log2(startIndex), np.log2(endIndex), int(1+np.log2(endIndex) - np.log2(startIndex)), base=2, dtype="int")

        M = constant
        h = 1/(M+1)

        eulerError = UMR(M*np.ones_like(timesteps), timesteps, "be", plot=plot)
        crankError = UMR(M*np.ones_like(timesteps), timesteps, "crank", plot=plot)
        
        dof = constant*timesteps
        
        expectedEuler = 1
        expectedCrank = 2
    
    #3. Refinement of both h and k, with c=k/h constant
    if typ=="c-const":
        
        M = np.logspace(np.log2(startIndex), np.log2(endIndex), int(1+np.log2(endIndex) - np.log2(startIndex)), base=2, dtype="int")
        
        h=1/(M+1)
        k=constant*h
        timesteps = np.array((1.0/k).round(), dtype="int")
        
        eulerError = UMR(M, timesteps, "be", plot=plot)
        crankError = UMR(M, timesteps, "crank", plot=plot)
        
        dof = timesteps*M
        
        expectedEuler = 0.5
        expectedCrank = 1
        
    
    #4. Refinement of both h and k, with r=k/h^2 constant 
    if typ == "r-const":
        
        M = np.logspace(np.log2(startIndex), np.log2(endIndex), int(1+np.log2(endIndex) - np.log2(startIndex)), base=2, dtype="int")

        h=1/(M+1)
        k=constant*h**2
        timesteps = np.array((1.0/k).round(), dtype="int")
        
        eulerError = UMR(M, timesteps, "be", plot=plot)
        crankError = UMR(M, timesteps, "crank", plot=plot)
        
        dof = timesteps*M**2
        
        expectedEuler = 0.66
        expectedCrank = 0.66

    fig, ax = plt.subplots()
    ax.loglog(dof, eulerError["continuous"], label="$L_2$ error", marker=".")
    ax.loglog(dof, eulerError["discrete"], label="$\ell_2$ error")
    ax.loglog(*expectedOrder(dof[0], dof[-1], eulerError["continuous"][0], expectedEuler), label="Expected order " + str(expectedEuler), linestyle="dashed")
    print("\nOrder of euler with UMR in x-direction", estimateOrder(dof, eulerError["continuous"]),"\n")
    ax.legend()
    ax.set_xlabel("Degrees of freedom")
    ax.set_ylabel("Relative error $e_{(\cdot)}^r$")
    ax.grid()
    plt.show()

    fig, ax = plt.subplots()
    ax.loglog(dof, crankError["continuous"], label="$L_2$ error", marker=".")
    ax.loglog(dof, crankError["discrete"], label="$\ell_2$ error")
    ax.loglog(*expectedOrder(dof[0], dof[-1], crankError["continuous"][0], expectedCrank), label="Expected order " + str(expectedCrank), linestyle="dashed")
    ax.legend()
    ax.set_xlabel("Degrees of freedom")
    ax.set_ylabel("Relative error $e_{(\cdot)}^r$")
    ax.grid()
    plt.show()
    print("Order of CN with UMR: ", estimateOrder(dof, crankError["continuous"]),"\n")